## MODEL Building

### Over sampling dataset 

In [3]:
#importing modules
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_curve, auc

# Importing and reading data

In [4]:
#Defining path for oversampling data with nan columns droped
path=Path('Resources/Cleaned_data/over_samp_col.csv')

In [6]:
#importing over sampling dataset
df = pd.read_csv(path)
#looking the few rows
df.head()

,name,dob,houseID,houseZip,paymentDate,paymentAmount,rentAmount,age,city,county,state,status
0,Karima Germany,1951-05-23,1192,92154,1,1321.0,1321.0,71,San Diego,San Diego County,CA,low risk
1,Agustina Spargo,1900-01-01,21,92111,1,2289.0,2289.0,122,San Diego,San Diego County,CA,low risk
2,Lucilla Broderick,1900-01-01,1474,92159,1,1439.0,1439.0,122,San Diego,San Diego County,CA,low risk
3,Russ Mchale,1977-04-20,2015,92137,1,1744.0,1744.0,45,San Diego,San Diego County,CA,low risk
4,Carmelita Ritzer,1969-03-09,311,92136,1,1471.0,1471.0,53,San Diego,San Diego County,CA,low risk


In [7]:
# looking the status columns
df.status.value_counts()

low risk     52640
high risk    52640
Name: status, dtype: int64

## Since many has date of birth as 1900-01-01, which is missing date of birth. 
### As missing date of birth cannot be gussed we neeed to remove it.

In [8]:
# Filtering the age between 1 and 100
df=df.loc[(df['age']>1) & (df['age']<100)]

In [9]:
#looking the few rows
df.head()

,name,dob,houseID,houseZip,paymentDate,paymentAmount,rentAmount,age,city,county,state,status
0,Karima Germany,1951-05-23,1192,92154,1,1321.0,1321.0,71,San Diego,San Diego County,CA,low risk
3,Russ Mchale,1977-04-20,2015,92137,1,1744.0,1744.0,45,San Diego,San Diego County,CA,low risk
4,Carmelita Ritzer,1969-03-09,311,92136,1,1471.0,1471.0,53,San Diego,San Diego County,CA,low risk
5,Clifton Ellwood,1993-11-02,430,92103,1,1233.0,1233.0,29,San Diego,San Diego County,CA,low risk
8,Lashawn Rotella,1985-12-05,1336,92190,1,1930.0,1930.0,37,San Diego,San Diego County,CA,low risk


In [11]:
# looking the status columns after dropping nan date of birth
df["status"].value_counts()

low risk     26335
high risk    25670
Name: status, dtype: int64

In [12]:
#shape of dataframe
df.shape

(52005, 12)

In [13]:
#Columns of the dataframe
df.columns

Index(['name', 'dob', 'houseID', 'houseZip', 'paymentDate', 'paymentAmount',
       'rentAmount', 'age', 'city', 'county', 'state', 'status'],
      dtype='object')

# Making feature matrix and target

In [14]:
# Only selecting important columns for machine learning
rent_df=df[[ 'houseZip', 'paymentDate', 'paymentAmount','rentAmount', 'age', 'status']]

In [15]:
# Creating feature matrix (X) by selecting all columns except 'status'
X=rent_df.drop('status', axis=1) 

# Creating response vector (y) by selecting 'status' column and converting to numpy
y=rent_df['status'].values 

In [16]:
#looking the few rows of X datafrmae
X.head()

,houseZip,paymentDate,paymentAmount,rentAmount,age
0,92154,1,1321.0,1321.0,71
3,92137,1,1744.0,1744.0,45
4,92136,1,1471.0,1471.0,53
5,92103,1,1233.0,1233.0,29
8,92190,1,1930.0,1930.0,37


# Converting the categorical data to binary

In [18]:
# Using pd.get_dummies to the zipcode 
X_dummies = pd.get_dummies(X, columns = ['houseZip'])

# Columns of X after dummies
X_dummies.columns


Index(['paymentDate', 'paymentAmount', 'rentAmount', 'age', 'houseZip_92101',
       'houseZip_92102', 'houseZip_92103', 'houseZip_92104', 'houseZip_92105',
       'houseZip_92106', 'houseZip_92107', 'houseZip_92108', 'houseZip_92109',
       'houseZip_92110', 'houseZip_92111', 'houseZip_92112', 'houseZip_92113',
       'houseZip_92114', 'houseZip_92115', 'houseZip_92116', 'houseZip_92117',
       'houseZip_92119', 'houseZip_92120', 'houseZip_92121', 'houseZip_92122',
       'houseZip_92123', 'houseZip_92124', 'houseZip_92126', 'houseZip_92127',
       'houseZip_92128', 'houseZip_92129', 'houseZip_92130', 'houseZip_92131',
       'houseZip_92132', 'houseZip_92133', 'houseZip_92134', 'houseZip_92135',
       'houseZip_92136', 'houseZip_92137', 'houseZip_92138', 'houseZip_92139',
       'houseZip_92140', 'houseZip_92142', 'houseZip_92145', 'houseZip_92147',
       'houseZip_92149', 'houseZip_92150', 'houseZip_92152', 'houseZip_92153',
       'houseZip_92154', 'houseZip_92155', 'houseZip_

In [22]:
#Total columns before and after get_dummies
print(f'Total column before: {X.shape[1]}\nTotal columns after: {X_dummies.shape[1]}')

Total column before: 5
Total columns after: 85


In [23]:
# Converting output labels to 0 and 1
y_label = LabelEncoder().fit_transform(y)
y_label

array([1, 1, 1, ..., 0, 0, 0])

# Spliting data for training set and testing set
###  Training set: Used to train the classifier. (80 percent of the data)
###  Testing set: Used to estimate the error rate of the trained classifier. (20 percent of data)

In [26]:
# Split using train test split 
X_train, X_test, y_train, y_test = train_test_split(X_dummies, y_label,stratify=y, random_state=42)

# Scaling the data to the same scale

In [27]:
# Using StandardScalr from scikit-lean for scaling the data
scaler = StandardScaler().fit(X_train)

In [28]:
# Scaling the X data by using StandardScaler()
X_train_scaled = scaler.transform(X_train)
X_train_scaled

array([[ 0.63906686, -3.17511235, -1.10547531, ..., -0.09488265,
        -0.09945347, -0.09692476],
       [ 0.63906686,  0.40932804,  0.73073554, ..., -0.09488265,
        -0.09945347, -0.09692476],
       [-1.56478149,  0.31821575, -0.11674639, ..., -0.09488265,
        -0.09945347, -0.09692476],
       ...,
       [-1.56478149,  0.14634483, -0.34787782, ..., -0.09488265,
        -0.09945347, -0.09692476],
       [ 0.63906686, -0.06279929, -0.43776227, ..., -0.09488265,
        -0.09945347, -0.09692476],
       [ 0.63906686, -0.3382069 , -0.42813179, ..., -0.09488265,
        -0.09945347, -0.09692476]])

In [29]:
# Transforming the test dataset 
X_test_scaled = scaler.transform(X_test)
X_test_scaled

array([[ 0.63906686,  0.24159859,  0.47071268, ..., -0.09488265,
        -0.09945347, -0.09692476],
       [ 0.63906686,  1.21691426,  0.62159014, ..., -0.09488265,
        -0.09945347, -0.09692476],
       [-1.56478149, -0.47487534, -1.18572928, ..., -0.09488265,
        -0.09945347, -0.09692476],
       ...,
       [ 0.63906686, -0.52664369, -0.72025625, ..., -0.09488265,
        -0.09945347, -0.09692476],
       [ 0.63906686, -0.71508047, -1.35907785, ..., -0.09488265,
        -0.09945347, -0.09692476],
       [ 0.63906686,  1.32045096,  1.51080413, ..., -0.09488265,
        -0.09945347, -0.09692476]])

# Making function for Model accuracy 
## Function that use only training set
## Function that use both traingin set and testing set ( Only used at the last of ml section)

In [30]:
#defining function for model scoring with training set only
def training_score(model, X_train_scaled=X_train_scaled, y_train=y_train):
    #Printing the accuracy score
    train_score=model.score(X_train_scaled, y_train)
    print(f'The training score of {model} model is {train_score}')
    print()
    print()
    # Using cross validation 
    cvs=cross_val_score(model,X_train_scaled, y_train,cv=5, scoring='accuracy' )
    print(f'Using k=5 cross validation the accuracy is {cvs}')
    print(f'Using k=5 cross validation the mean accuracy is {np.mean(cvs)}')
    print()
    print()
    #predection on the test set and confusion matrix
    y_train_pred=cross_val_predict(model,X_train_scaled, y_train,cv=3)
    cm=confusion_matrix(y_train, y_train_pred)
    print(f'Confusion Matrix using Cross validation predict is:\n {cm}')
    print()
    print()
    #classification report
    cr=classification_report(y_train, y_train_pred)
    print(f'Classification report is:\n {cr}')
    print()
    print()
    #Roc curve
    y_train_predc=cross_val_predict(model,X_train_scaled, y_train,cv=5,method="predict_proba")
    y_proba = y_train_predc[:,1]  #proba of positive class
    fpr, tpr, thresholds = roc_curve(y_train, y_proba)
    # Plotting the roc curve
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange')
    plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f"roc curve for {model}")
    print(f"The ROC Curve using {model} model")
    plt.show()
    print()
    # Area under the curve (AUC)
    auc = auc(fpr, tpr)
    print(f'The are under the curve is: {auc}')


In [31]:
#defining function for model scoring with test set
def train_test_score(model, X_train_scaled=X_train_scaled, X_test_scaled=X_test_scaled, 
               y_train=y_train,y_test=y_test):
    #Accuracy score
    train_score=model.score(X_train_scaled, y_train)
    test_score=model.score(X_test_scaled, y_test)
    print(f'The training score of {model} model is {train_score}')
    print(f'The testing score of {model} model is {test_score}')
    print()
    print()
    #Confusion matrix
    y_pred=model.predict(X_test_scaled)
    cm=confusion_matrix(y_test, y_pred)
    print(f'Confusion Matrix by predicting using test data is:\n {cm}')
    print()
    print()
    #classification report
    cr=classification_report(y_test, y_pred)
    print(f'Classification report is:\n {cr}')
    print()
    print()
    #roc curve
    y_proba = model.predict_proba(X_test_scaled)[:,1]  #proba of positive class
    fpr, tpr, thresholds = roc_curve(y_test, y_proba)
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange')
    plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f"roc curve for {model}")
    print(f"The ROC Curve using {model} model")
    plt.show()
    # Area under the curve (AUC)
    roc_auc = auc(fpr, tpr)
    print()
    print(f'The are under the curve is: {auc}')

In [33]:
#defining function for model scoring 
def model_score(model, X_train_scaled=X_train_scaled, X_test_scaled=X_test_scaled, 
               y_train=y_train,y_test=y_test):
    #training score
    train_score=model.score(X_train_scaled, y_train)
    #testing score
    test_score=model.score(X_test_scaled, y_test)
    #prediction on test data
    y_pred=model.predict(X_test_scaled)
    #Precission score
    Precision_score=precision_score(y_test, y_pred)
    #Recall
    Recall_score=recall_score(y_test, y_pred)
    #f1-score
    F1_score=f1_score(y_test, y_pred)
    #Area under the curve
    y_proba = model.predict_proba(X_test_scaled)[:,1]  #proba of positive class
    fpr, tpr, thresholds = roc_curve(y_test, y_proba)
    
    roc_auc = auc(fpr, tpr)
    result=[train_score, test_score, Precision_score, Recall_score,F1_score, roc_auc ]
    return result